# Lamprey Transcriptome Analysis: Genome Completeness Assessment

```
Camille Scott [camille dot scott dot w @gmail.com] [@camille_codon]

camillescott.github.io

Lab for Data Intensive Biology (DIB)
University of California Davis
```

## About

Uses the many-sample de novo transcriptome assembly to attempt to assess the completeness of the genome build (and orthogonally, the completeness of the transcriptome itself).

    assembly version: lamp10

    assembly program: Trinity
    
    genome build: 7.0.75 (ensembl release 75

## Libraries

In [3]:
%load_ext autoreload
%autoreload 2
%autosave 60

Autosaving every 60 seconds


In [4]:
from libs import *
%run -i common.ipy

** Using data resources found in ../resources.json
** Using config found in ../config.json


In [5]:
import pyprind

In [6]:
from blasttools import blast_to_df

In [7]:
from IPython.display import display, Image
import glob

In [8]:
%pylab inline
from matplotlib import rc_context
tall_size = (8,16)
norm_size = (12,8)
mpl_params = {'figure.autolayout': True,
               'axes.titlesize': 24,
               'axes.labelsize': 16,
               'ytick.labelsize': 14,
               'xtick.labelsize': 14
               }
sns.set(style="white", palette="Paired", rc=mpl_params)
#sns.set_palette("Paired", desat=.6)
b, g, r, p = sns.color_palette("muted", 4)

Populating the interactive namespace from numpy and matplotlib


In [9]:
from ete3 import NCBITaxa

In [10]:
%config InlineBackend.close_figures = False

## Data

We'll save our results to a dictionary and dump it to JSON for use in the paper.

In [11]:
results = {}

In [12]:
store = pd.HDFStore(wdir('{}.store.h5'.format(prefix)), complib='zlib', complevel=5)

In [13]:
import atexit

In [14]:
def dump_results(fn='../doc/petmar-genome-comp.results.json'):
    with open(fn, 'wb') as fp:
        json.dump(results, fp)

In [15]:
def exit_func():
    dump_results()
    store.close()
atexit.register(exit_func)

<function __main__.exit_func>

## Orthologs

How many transcripts have orthologs in other organisms, but no lamprey support? And vice versa?

In [16]:
ortho_panel = store['lamp10_ortho']

In [17]:
ortho_df = store['lamp10_ortho_filter_df']

In [18]:
blast_df = store['lamp10_blast_filter_df']

In [19]:
# Transcripts with recipricol best thits to our main databases
has_ortho = ((ortho_df['lamp10.fasta.x.braFlo.pep.all.fa.db.tsv'] == True) | 
             (ortho_df['lamp10.fasta.x.danRer.pep.fa.db.tsv'] == True) |
             (ortho_df['lamp10.fasta.x.homSap.pep.fa.db.tsv'] == True) |
             (ortho_df['lamp10.fasta.x.musMus.pep.fa.db.tsv'] == True))

In [20]:
# We want to filter out transcripts with *any* homologies to the genome,
# the lamprey proteins, and the lamprey cDNAs (lamp0)
lamp_filter = ((blast_df['lamp10.fasta.x.petMar2.fa.db.tsv'] == False) & 
               (blast_df['lamp10.fasta.x.petMar2.pep.fa.db.tsv'] == False) &
               (blast_df['lamp10.fasta.x.petMar2.cdna.fa.db.tsv'] == False))

In [21]:
ortho_df = ortho_panel.minor_xs('sseqid')

In [22]:
n_novel = (has_ortho & lamp_filter).sum()
results['n_novel_ortho'] = n_novel
print n_novel, 'have orthologies but no lamprey support'

1768 have orthologies but no lamprey support


In [23]:
n_genome_supported = (has_ortho & (blast_df['lamp10.fasta.x.petMar2.fa.db.tsv'] == True)).sum()
results['n_genome_supported'] = n_genome_supported
print n_genome_supported, 'have orthologies and genome support'

11990 have orthologies and genome support


In [24]:
n_supported = (has_ortho & (lamp_filter == False)).sum()
results['n_supported_ortho'] = n_supported
print n_supported, 'have orthologies and lamprey support'

13405 have orthologies and lamprey support


In [25]:
print 'This means {:.2f}% of orthologs are potentially novel'.format((float(n_novel) / (n_novel + n_supported)) * 100.0)

This means 11.65% of orthologs are potentially novel


### Corresponding Genes

In [52]:
GNATHOSTOMATA = 7776
CYCLOSTOMATA = 1476529
CEPHALOCHORDATA = 7735 
PETMAR = 7757
BRAFLO = 7739
MUSMUS = 10090
HOMSAP = 9606
DANRER = 7955
MYXINIDAE = 7762

In [30]:
mg = mygene.MyGeneInfo()

Fix the uniprot names for querying

In [32]:
ortho_df['lamp10.fasta.x.Myx.pep.all.fa.db.tsv'] = ortho_df['lamp10.fasta.x.Myx.pep.all.fa.db.tsv'].apply(uniprot_str)

In [33]:
ortho_df['lamp10.fasta.x.braFlo.pep.all.fa.db.tsv'] = ortho_df['lamp10.fasta.x.braFlo.pep.all.fa.db.tsv'].apply(uniprot_str)

Make a DataFrame of just the potentially novel transcripts' orthology names

In [34]:
novel_df = ortho_df[has_ortho & lamp_filter]

In [138]:
def classify_orthology(df):
    '''
    Given a DataFrame containing the names of transcript orthologies (either uniprot or ensembl identifiers),
    determine whether each orthology corresponds to a gene in the cyclostomata, cephalochordata, or 
    gnathostomata lineages. Returns a list of tuples of booleans with the results, of the form:
    
    [(in cephalochordata, in gnathostomata, in cyclostomata),
     (...),
     ...]
     
     WARNING: This uses the mygene API and makes many complex queries; accordingly, it's slow as hell
    '''
    species=','.join(str(s) for s in [BRAFLO, MUSMUS, HOMSAP, DANRER, MYXINIDAE])
    classifications = []
    for n, (tr_id, row) in enumerate(df.iterrows()):
        if n % 1000 == 0:
            print 'Classified {n}...'.format(n=n)
        queries = [q for q in list(row) if type(q) is str]
        results = mg.querymany(queries, scopes='uniprot,ensemblprotein,ensembltranscript', 
                               species=species, as_dataframe=True, verbose=False)
        try:
            # remove failed queries, if there are any
            results = results[results['notfound'] != True]
        except KeyError:
            pass
        
        def check_results(res_list):
            for res in res_list:
                if 'notfound' not in res:
                    return True
            return False
        
        # Check if any are in the cephalochorate lineage
        try:
            ceph_results = mg.querymany(list(results.symbol), scopes='symbol', fields='taxid', 
                                    species=CEPHALOCHORDATA, include_tax_tree=True, verbose=False)
            ceph = check_results(ceph_results)
        except AttributeError:
            ceph = False
        
        # Check cyclostomata lineage
        try:
            cycl_results = mg.querymany(list(results.symbol), scopes='symbol', fields='taxid', 
                                    species=CYCLOSTOMATA, include_tax_tree=True, verbose=False)
            cycl = check_results(cycl_results)
        except AttributeError:
            cycl = False
        
        # Check gnathostomata lineage
        try:
            gnat_results = mg.querymany(list(results.symbol), scopes='symbol', fields='taxid', 
                                    species=GNATHOSTOMATA, include_tax_tree=True, verbose=False)
            gnat= check_results(gnat_results)
        except AttributeError:
            gnat = False
        
        classifications.append((ceph, gnat, cycl))
    return classifications

In [130]:
ortho_classes = classify_orthology(novel_df)

Classified 0...
Classified 1000...


In [136]:
class_df = pd.DataFrame(ortho_classes, columns=['cephalochordata', 'gnathostomata', 'cyclostomata'],
                        index=novel_df.index)

In [139]:
class_df.head()

,cephalochordata,gnathostomata,cyclostomata
target_id,,,
c366614_g4_i7,True,False,False
c329845_g1_i1,False,True,False
c363162_g3_i1,True,True,False
c351306_g1_i1,False,True,False
c351306_g1_i2,True,True,False


In [144]:
class_df = pd.merge(novel_df, class_df, left_index=True, right_index=True)
del class_df['lamp10.fasta.x.petMar2.pep.fa.db.tsv']
del class_df['lamp10.fasta.x.petMar2.ncrna.fa.db.tsv']
del class_df['lamp10.fasta.x.petMar2.cds.fa.db.tsv']
del class_df['lamp10.fasta.x.petMar2.cdna.fa.db.tsv']
del class_df['lamp10.fasta.x.petMar2.fa.db.tsv']

Orthologies specific to the gnathostomata or cyclostomata line -- they're in either gnathostomata or cyclostomata, but not in cephalochordata.

In [151]:
(((class_df.gnathostomata == True) | (class_df.cyclostomata == True)) & (class_df.cephalochordata == False)).sum()

796

In [162]:
((class_df.gnathostomata == True) & (class_df.cephalochordata == False)).sum()

796

It looks like it makes no difference whether we include cyclostomata. That is, we actually have up to 796 orthologies which are specific to jawed vertebrates but probably not a part of any existing cyclostome database. Cool! Given that, let's check what happens when we enforce both -- that is, genes which are specific to the vertebrate line.

In [165]:
((class_df.cyclostomata == True) & (class_df.gnathostomata == True) & (class_df.cephalochordata == False)).sum()

60

So 60 of those orthologies have cyclostomata support -- they likely aren't in sea lamprey resources, so they must either be from hagfish or another lamprey. How about when we exclude the entire cyclostomata lineage?

In [166]:
((class_df.cyclostomata == False) & (class_df.gnathostomata == True) & (class_df.cephalochordata == False)).sum()

736

Here's what we're really trying to get at -- that's 736 orthologies which are specific to the jawed vertebrate line according to current resources, but clearly have pretty strong support in our transcriptome. Alright, what are these genes?

In [170]:
novel_df[((class_df.cyclostomata == False) & (class_df.gnathostomata == True) & (class_df.cephalochordata == False))].head()

,lamp10.fasta.x.Myx.pep.all.fa.db.tsv,lamp10.fasta.x.braFlo.pep.all.fa.db.tsv,lamp10.fasta.x.danRer.pep.fa.db.tsv,lamp10.fasta.x.homSap.pep.fa.db.tsv,lamp10.fasta.x.musMus.pep.fa.db.tsv,lamp10.fasta.x.petMar2.fa.db.tsv,lamp10.fasta.x.petMar2.cdna.fa.db.tsv,lamp10.fasta.x.petMar2.cds.fa.db.tsv,lamp10.fasta.x.petMar2.ncrna.fa.db.tsv,lamp10.fasta.x.petMar2.pep.fa.db.tsv
target_id,,,,,,,,,,
c329845_g1_i1,NaN,NaN,ENSDARP00000105046,ENSP00000329735,ENSMUSP00000040433,NaN,NaN,NaN,NaN,NaN
c351306_g1_i1,NaN,NaN,NaN,NaN,ENSMUSP00000136481,NaN,NaN,NaN,NaN,NaN
c292244_g1_i1,NaN,NaN,ENSDARP00000018388,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c360287_g1_i1,NaN,NaN,ENSDARP00000009179,ENSP00000256997,ENSMUSP00000108209,NaN,NaN,NaN,NaN,NaN
c265851_g1_i1,NaN,NaN,NaN,ENSP00000443978,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
def get_symbols(df):
    species=','.join(str(s) for s in [BRAFLO, MUSMUS, HOMSAP, DANRER, MYXINIDAE])
    queries = [q for q in set(df.values.flatten()) if type(q) is str]
    results = mg.querymany(queries, scopes='uniprot,ensemblprotein,ensembltranscript', 
                               species=species, as_dataframe=True, verbose=False)
    symbols = set([r.upper() for r in results.symbol if type(r) in [str,unicode]])
    return symbols

In [191]:
jvert_specific = get_symbols(novel_df[((class_df.cyclostomata == False) & (class_df.gnathostomata == True) & 
                                       (class_df.cephalochordata == False))])

In [193]:
len(jvert_specific)

780

In [197]:
(class_df.sum(axis=1) == 0).sum()

103

## Germline DNA Samples

We've got some nice shiny new sperm DNA samples - let's play with them some. First things first, we'll check out their FastQC results to make sure they're not crap.

In [30]:
fastqc_folders = sorted(glob.glob('*fastqc'))

In [31]:
!ls 2_ATCACG_L001_R1_001_fastqc/Images

duplication_levels.png	 per_base_sequence_content.png
kmer_profiles.png	 per_sequence_gc_content.png
per_base_gc_content.png  per_sequence_quality.png
per_base_n_content.png	 sequence_length_distribution.png
per_base_quality.png


In [32]:
# quick utility function to generate a table of images
def get_img_table(folders, image):
    htmlout = '<table>'
    for i in xrange(0, len(folders), 2):
        left = fastqc_folders[i]
        right = fastqc_folders[i+1]
        htmlout += '<tr>'
        htmlout += '<td align="center">' + left + ', LEFT</td>'
        htmlout += '<td align="center">' + right + ', RIGHT</td>'
        htmlout += '</td>'
        
        htmlout += '<tr>'
        htmlout += '<td><img src="' + os.path.join(left, 'Images', image) + '"/></td>'
        htmlout += '<td><img src="' + os.path.join(right, 'Images', image) + '"/></td>'
        htmlout += '</tr>'
    htmlout += '</table>'
    return htmlout

#### Per-Base Sequence Quality

These look pretty normal -- in fact, the right fragments are a little better than I might expect for many runs.

In [33]:
display(HTML(get_img_table(fastqc_folders, 'per_base_quality.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


#### GC Content

For the most part, these results line up with what we expect from the lamprey genome paper: ~46% GC content for the whole genome (FastQC throws a warning for this check, but we can safely ignore it based on our prior knowledge). Curiously, in the genome paper, they report that coding regions had a GC content of ~61%, but we don't really see a bump in the distribution there -- instead, we see a bump at ~84% in all samples. There isn't anything unexpected in the per-*base* figures, so we can at least assume there isn't a technical artifact common to all reads.

In [34]:
display(HTML(get_img_table(fastqc_folders, 'per_sequence_gc_content.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


In [35]:
display(HTML(get_img_table(fastqc_folders, 'per_base_gc_content.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


Once more, nothing all too exciting -- a bit of A bias toward the end of the righ reads, which coincides with the expected drop in quality.

In [36]:
display(HTML(get_img_table(fastqc_folders, 'per_base_sequence_content.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


Seems a little odd that we get this slow increase in G/C homopolymers over the length of our reads. Need to investigate further.

In [37]:
display(HTML(get_img_table(fastqc_folders, 'kmer_profiles.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


In [38]:
display(HTML(get_img_table(fastqc_folders, 'duplication_levels.png')))

"2_ATCACG_L001_R1_001_fastqc, LEFT","2_ATCACG_L001_R2_001_fastqc, RIGHT"
,
"2_TGACCA_L001_R1_001_fastqc, LEFT","2_TGACCA_L001_R2_001_fastqc, RIGHT"
,
"4_CCGTCC_L001_R1_001_fastqc, LEFT","4_CCGTCC_L001_R2_001_fastqc, RIGHT"
,
"4_GATCAG_L001_R1_001_fastqc, LEFT","4_GATCAG_L001_R2_001_fastqc, RIGHT"
,


Summary: we should probably run a trimmer on these samples before using them. Shocker!